In [1]:
batch_size_for_classification = 8000
#prediction
prediction_path = "Export/cdnow/cdnow_prediction_variantB_"+str(batch_size_for_classification+2000)+"_clusterwise.csv"

In [2]:
import numpy as np
import pandas as pd
from functools import reduce
from joblib import Parallel, delayed
from sklearn.base import clone
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ParameterGrid
import networkx as nx
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from trmf import TRMFRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import datetime
import random
import rpy2
import math
import trmf
from sklearn.model_selection import KFold, GridSearchCV
import time
import pickle
import warnings
warnings.filterwarnings("ignore")
from rpy2.robjects.packages import importr
import rpy2.robjects.packages as rpackages
import rpy2.robjects as robjects
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1) # select the first mirror in the list
packnames = ['forecast']
from rpy2.robjects.vectors import StrVector
try:
    base = importr('forecast')
except:
    utils.install_packages(StrVector('forecast'))
    base = importr('forecast')

In [3]:
# Path fo import files
#input data
import_data = "import/CDNOW_master.txt"

#Clusters
import_clusters = "clusters/results_cdnow_ensembles.csv"

### 0. Preprocessing cdnow

In [4]:
df = pd.read_csv(import_data)

In [5]:
df = pd.read_csv(import_data,header=None,names  =['id','Date','disks','price_purchase'],delim_whitespace=True,)
df['Date'] = df["Date"].apply(lambda x:datetime.datetime.strptime(str(x),"%Y%m%d"))
df = df.drop(columns = ['disks'])
df = df.groupby(by=['id','Date']).sum().reset_index()
df = df.pivot(columns = 'Date',index = 'id',values = 'price_purchase')
df.sample(2)

Date,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,1997-01-08 00:00:00,1997-01-09 00:00:00,1997-01-10 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
8045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 0.1 Choose TS with given clusters and divide into test/train dataset

In [6]:
cluster_TS  = pd.read_csv(import_clusters)
cluster_TS.head()

,Unnamed: 0,clusters_r,clusters_m,clusters_f,GMM_Voting,GMM_Pair
0,11770,0,0,0,2,6
1,14675,17,0,8,5,5
2,10026,2,0,3,0,3
3,13997,1,0,2,3,5
4,741,0,0,0,2,1


In [7]:
df_train = cluster_TS.merge(df,left_on = "Unnamed: 0", right_on = 'id')

In [8]:
df_train = df_train.rename(index = str, columns={"Unnamed: 0": "id"}).set_index('id')
df_train.head()

,clusters_r,clusters_m,clusters_f,GMM_Voting,GMM_Pair,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
11770,0,0,0,2,6,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14675,17,0,8,5,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10026,2,0,3,0,3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13997,1,0,2,3,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741,0,0,0,2,1,NaN,NaN,NaN,43.13,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# for this we assign clusters
df_test= df.drop(df_train.index.values,axis=0).sample(batch_size_for_classification)
df_test.head()

Date,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,1997-01-08 00:00:00,1997-01-09 00:00:00,1997-01-10 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
5780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### NAN -> 0

In [10]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)
df_train.sample(2)

,clusters_r,clusters_m,clusters_f,GMM_Voting,GMM_Pair,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
4162,0,0,0,2,6,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3789,0,0,12,2,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 1. Prediction with given clusters on train data (2000 rows)

In [11]:
train_targets, test_targets= train_test_split(df_train.transpose(), test_size=.127, shuffle=False)  #.10 also has to be changed below(classifer), TODO  add var
train_targets = train_targets.transpose()
test_targets = test_targets.transpose()

In [12]:
# # columsn to predict
test_targets.shape[1]

70

In [13]:
# output file
result = pd.DataFrame()

### 1.0.1 GridSearch function for TMRF

In [14]:
base = TRMFRegressor(n_components=1,
                 n_order=0,
                 fit_regression=False,
                 fit_intercept=True,
                 nonnegative_factors=True,
                 n_max_mf_iter=5)


def gridsearch_trmf(X,test_targets):
    grid = ParameterGrid(dict(
    n_components=np.r_[1:2],#17],
    n_order=np.r_[1:2],#17],
    C_Z=np.logspace(-2, 1,num=1),#,num=4),
    C_F=np.logspace(-2, 1,num=1),#,num=4),
    C_phi=np.logspace(-2, 1,num=1),#,num=4),
    eta_Z=np.linspace(0.05, 0.95,num=1)#,num=10),
    ))
    
    transformer = StandardScaler(with_mean=True, with_std=True)
    X = transformer.fit_transform(X)

    
    # run the experiment in parallel
    par_ = Parallel(n_jobs=-1, verbose=0)
    results = par_(delayed(helper)(par, X, test_targets.transpose(),transformer) for par in grid)
    
    keys = ['n_order', 'n_components', 'eta_Z', 'C_phi', 'C_Z', 'C_F']
    data = dict((tuple(par[k] for k in keys), rmse,) for par, rmse in results)
    sr = pd.Series(data, name="rmse").sort_index().rename_axis(keys)
    cube = sr.values.reshape(*[len(grid.param_grid[0][k]) for k in keys])
    stepping = [grid.param_grid[0][k] for k in keys]
    # find the flat index of the smallest value
    flat_index = np.argmin(cube)
    # ... and unravel into into a multidimensional index
    index = np.unravel_index(flat_index, cube.shape)
    # collect the best paramaters from the grid
    best_ = {k: grid.param_grid[0][k][i] for k, i in zip(keys, index)}
    return best_

transformer = StandardScaler(with_mean=True, with_std=True)

def helper(par, train, test,  transformer,base=base):
    basee = TRMFRegressor(n_components=1,
                     n_order=0,
                     fit_regression=False,
                     fit_intercept=True,
                     nonnegative_factors=True,
                     n_max_mf_iter=5)
    
    # clone, set parameters and fit
    trmf = clone(basee).set_params(**par).fit(train)
    
    # predict and return
    pred = transformer.inverse_transform(
        trmf.predict(n_ahead=len(test)))

    return par, mean_squared_error(test, pred)

## 1.0.2 TMRF with clusterwise regression

In [15]:
def clusterwise_regression(X,Y,
                           cluster_column ='supercluster',
                           columns_wo_TS = ['clusters_r','clusters_m','clusters_f','supercluster'],
                           gridsearch_limit = 1):
    """
    Runs clusterwise regression
    Parameters
    ----------
    X : DataFrame with train data (rows: id; columns: clusters + TS)
    Y : DataFrame with true_values (rows: id; columns: clusters + TS) (to compare gggpredictions)
    cluster_column : column with given clusters
    columns_wo_TS  : columns in DataFrames not regarded as TimeSeries
    gridsearch_limit : # of times to run GridSeach for each cluster (>=1)
    ---------
    Return :
    clusters : Dictionary with trained models for each cluster
    old_cluster_for_user : Dictionary with optimal cluster for each user (after moving)
    cluster_parameters_for_gridsearch : Dictionary with optimal parameters for each cluster model
    """
    train_targets = X.copy()
    test_targets = Y.copy()
    n_ahead = test_targets.shape[1]
    clusters = {}
    # save cluster info for clusterwise regression
    cluster_error = {}
    cluster_has_changed = {}
    cluster_data_X = {}
    counter_for_gridsearch = 0
    old_clusters ={}
    cluster_data_true_values = {}
    cluster_predicted = {}
    score_for_user = {}
    cluster_parameters_for_gridsearch = {}
    previous_total_error = math.inf
    limit = 3
    for i in list(range(0,limit+1)): # Some limit on iterations just in case
        # Assign user to clusters
        cluster_for_user = {}
        cluster_for_user  = train_targets[cluster_column].to_dict()
        for cluster_number in train_targets[cluster_column].unique():
            # Save training data for this cluster
            cluster_data_X[cluster_number] = train_targets.loc[train_targets[cluster_column] == cluster_number].drop(columns =columns_wo_TS)
            cluster_data_true_values[cluster_number] = test_targets.loc[ cluster_data_X[cluster_number].index]
            # Scaling (Not minmax because of it wasn't used in trmf github)
            transformer = StandardScaler(with_mean=True, with_std=True)
            cluster_data_X[cluster_number] = cluster_data_X[cluster_number].transpose()
            cluster_data_X[cluster_number] = transformer.fit_transform(cluster_data_X[cluster_number])
            #GridSearchCV for TRMF
            # If initial initialization or cluster has changed
            if (cluster_number not in cluster_has_changed) or (cluster_has_changed[cluster_number]== True):
                if (i<gridsearch_limit):
                    # add prev_cluster_paraeters
                    cluster_parameters_for_gridsearch[cluster_number] = gridsearch_trmf(cluster_data_X[cluster_number],cluster_data_true_values[cluster_number])
                clusters[cluster_number ]= TRMFRegressor(**cluster_parameters_for_gridsearch[cluster_number], eta_F=0., adj=None, C_B=0., fit_regression=False,
                                 fit_intercept=True, nonnegative_factors=True, n_max_mf_iter=5,
                                 z_step_kind="tron")
                clusters[cluster_number].fit(cluster_data_X[cluster_number])

            # As advised by Ivan
            fitted_ = np.dot(clusters[cluster_number].factors_,clusters[cluster_number].loadings_)+clusters[cluster_number].intercept_
            cluster_predicted[cluster_number] = transformer.inverse_transform( clusters[cluster_number].predict(fitted_,n_ahead=n_ahead))
            # Save prediction
            cluster_predicted[cluster_number] = pd.DataFrame(cluster_predicted[cluster_number]).transpose()
            cluster_predicted[cluster_number].columns  = cluster_data_true_values[cluster_number].columns 
            cluster_predicted[cluster_number].index = cluster_data_true_values[cluster_number].index
            #save cluster score
            cluster_error[cluster_number] = math.sqrt(mean_squared_error(cluster_data_true_values[cluster_number],cluster_predicted[cluster_number]))
        print("Error per cluster: ",cluster_error)
        # flatten true values
        all_true_values  = reduce(lambda x,y: list(x)+list(y), ([x.ravel() for x in [x.values for x in list(cluster_data_true_values.values())  ]])   )
        # flatten prediction 
        all_predictions  = reduce(lambda x,y: list(x)+list(y), ([x.ravel() for x in [x.values for x in list(cluster_predicted.values())  ]])   )
        total_error = math.sqrt(mean_squared_error(all_predictions,all_true_values))
        print("New error: ",total_error)
        # If new score is worse - load last cluster and run final prediction. 
        if (total_error > previous_total_error):
            train_targets[cluster_column] = list(old_cluster_for_user.values())
            print('Error has increased. Backtrack to previous iteration')
            clusters = old_clusters.copy()
            return clusters,old_cluster_for_user,cluster_parameters_for_gridsearch
        else:
            previous_total_error = total_error
        if (i>=limit):# return prev iteration
            return old_clusters.copy(),old_cluster_for_user,cluster_parameters_for_gridsearch
        # Calculate score per user
        for cluster_number in train_targets[cluster_column].unique():
            for index, row in cluster_data_true_values[cluster_number].iterrows():
                score_for_user[index] = math.sqrt(mean_squared_error( row, cluster_predicted[cluster_number].loc[index,:]))
        # Assume that we haven't change any cluster
        for x in cluster_has_changed:
            cluster_has_changed[x]=False        
        cluster_lowest_mae = min(cluster_error, key=cluster_error.get)
        old_clusters = {}
        old_cluster_for_user = {}
        ii =0
        #Take observation x from cluster a, where error_x > error_a
        for cluster_number in train_targets[cluster_column].unique():
            # All ids in this cluster are already in cluster with lowest possible mae
            if (cluster_number == cluster_lowest_mae):
                continue
            for index, row in cluster_data_true_values[cluster_number].iterrows():    
            # User is in current cluster -> we can reassign him
                if (cluster_number  == cluster_for_user[index]):
                    if (score_for_user[index] > cluster_error[cluster_number]):
                        ii+=1#counter
                        # Have to refit changed clusters             
                        cluster_has_changed[cluster_number] = True
                        cluster_has_changed[cluster_lowest_mae] = True
                        # Move observation x to the cluster with the lowest training erro
                        train_targets.loc[index,cluster_column] = cluster_lowest_mae   
        old_clusters = clusters.copy() # we have to return to old snapshot, if error didn't improve
        old_cluster_for_user = cluster_for_user.copy() # as well as save last assigned clusters                  
        print('Moved ', ii, ' users to another cluster \n')   
   # return old_clusters,old_cluster_for_user,cluster_parameters_for_gridsearch   

### 1.0.3 TRMF Prediction Function

In [16]:
def final_prediction_after_clusterwise_reg(
                           X,Y,
                           step = 'unstated',
                           clusters = None,
                           clusters_parameters = None,
                           new_clusters_for_users=None,
                           cluster_column ='supercluster',
                           columns_wo_TS = ['clusters_r','clusters_m','clusters_f','supercluster'],
                           gridsearch_use = False):
    """
    Performs final prediction and generates DataFrame for Alibaba Data
    Parameters
    ----------
    X : DataFrame with train data (rows: id; columns: clusters + TS)
    Y : DataFrame with true_values (rows: id; columns: clusters + TS) (to compare predictions)
    step : 'before_classifier' or 'after_classifier', for report generation
    clusters: dictionary with models for each cluster. If None, then refit
    clusters_parameters: dictionary with parameters for each cluster model
    new_clusters_for_users: dictionary with clusters for each user. 
    cluster_column : column with given clusters
    columns_wo_TS  : columns in DataFrames not regarded as TimeSeries
    gridsearch_use : Run gridSeach (True) or use clusters_parameters (False)?
    ---------
    Return :
    result : DataFrame with columns
    cluster, user id, true target, prediction, rmse, model, method (all data, clustered), step (all data, before classifier, after classifier)
    """
    result = pd.DataFrame()
    train_targets = X.copy()
    test_targets = Y.copy()
    n_ahead = test_targets.shape[1]
    if (new_clusters_for_users is not None):
        # we assume that the order in DataFrame is the same as in clusterwise regression
        train_targets[cluster_column]=list(cluster_per_user.values()) 
    cluster_data_X = {}
    cluster_data_true_values = {}
    if (clusters_parameters == None or gridsearch_use==True):
        clusters_parameters = {}
    if (clusters == None or clusters_parameters ==None):
        clusters = {}
    for cluster_number in train_targets[cluster_column].unique():
        cluster_data_X[cluster_number] = train_targets.loc[train_targets[cluster_column] == cluster_number].drop(columns =columns_wo_TS)
        cluster_data_true_values[cluster_number] = test_targets.loc[ cluster_data_X[cluster_number].index]
        # Scaling (Not minmax because of it wasn't used in trmf github)
        transformer = StandardScaler(with_mean=True, with_std=True)
        cluster_data_X[cluster_number] = cluster_data_X[cluster_number].transpose()
        cluster_data_X[cluster_number] = transformer.fit_transform(cluster_data_X[cluster_number])
        if (cluster_number not in clusters_parameters or gridsearch_use==True):
            clusters_parameters[cluster_number] = gridsearch_trmf(cluster_data_X[cluster_number],cluster_data_true_values[cluster_number])
        if (cluster_number not in clusters or ( cluster_number not in  clusters_parameters) or gridsearch_use==True):
            clusters[cluster_number ]= TRMFRegressor(**clusters_parameters[cluster_number], eta_F=0., adj=None, C_B=0., fit_regression=False,
                         fit_intercept=True, nonnegative_factors=True, n_max_mf_iter=5,
                         z_step_kind="tron")
            # If you feed clusters from clusterwise, you don't need to refit them
            clusters[cluster_number ].fit(cluster_data_X[cluster_number])
        fitted_ = np.dot(clusters[cluster_number].factors_,clusters[cluster_number].loadings_)+clusters[cluster_number].intercept_
        predicted= transformer.inverse_transform( clusters[cluster_number].predict(fitted_,n_ahead=n_ahead))
        df_prediction = pd.DataFrame(predicted).transpose()
        df_prediction.columns  = cluster_data_true_values[cluster_number].columns 
        df_prediction.index = cluster_data_true_values[cluster_number].index
        # Transform prediction for output
        #MODEL
        df_prediction.insert(0,'RMSE',math.sqrt(mean_squared_error(cluster_data_true_values[cluster_number],df_prediction)))
        df_prediction.insert(0,'model','trmf')
        #METHOD
        df_prediction.insert(0,'method','clustered_clusterwise')
        #STEP
        df_prediction.insert(0,'step',step)
        # CLUSTER
        df_prediction.insert(0,'cluster_id',str(cluster_column)+'_'+str(cluster_number))
        #Wide to logn
        df_prediction = df_prediction.reset_index().melt(id_vars =['id','cluster_id','step','method','model','RMSE']).rename(index = str, columns={'variable':'date','value':'predicted_value'})
        true_values=cluster_data_true_values[cluster_number].reset_index().melt(id_vars=['id']).rename(index = str, columns={'variable':'date','value':'true_value'})
        result = result.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))
    return result

### 1.0.4 Theta Prediction function

In [17]:
def theta_prediction(X,Y,step = 'unstated',
           cluster_column ='supercluster',
           columns_wo_TS = ['clusters_r','clusters_m','clusters_f','supercluster']):
    """
    Performs theta prediction
    Return :
    result : DataFrame with columns
    cluster, user id, true target, prediction, rmse, model, method (all data, clustered), step (all data, before classifier, after classifier)
    """
    train_targets = X.copy()
    test_targets = Y.copy()
    base = importr('forecast')
    ress = pd.DataFrame()
    for cluster_number in train_targets[cluster_column].unique():
        X = train_targets.loc[train_targets[cluster_column] == cluster_number].drop(columns =columns_wo_TS)
        true_values = test_targets.loc[ X.index]
        # Scaling
        scl =MinMaxScaler() #StandardScaler(with_mean=True, with_std=True)
        X = scl.fit_transform(X)
        #THETA
        df_prediction = pd.DataFrame(columns = test_targets.columns)
        train_targets_scaled = pd.DataFrame(X)
        train_targets_scaled.index = true_values.index
        for index, row in train_targets_scaled.iterrows():
            res = robjects.FloatVector(row.drop(columns = ['uid']))#to id
            r_prediction = base.thetaf(res,70)[1]
            df_prediction.loc[index] = r_prediction
        #Transform prediction for output 
        #MODEL
        df_prediction.insert(0,'RMSE',math.sqrt(mean_squared_error(true_values,df_prediction)))
        df_prediction.insert(0,'model','theta')
        #METHOD
        df_prediction.insert(0,'method','clustered')
        #STEP
        df_prediction.insert(0,'step',step)
        # CLUSTER
        df_prediction.insert(0,'cluster_id',str(cluster_column)+'_'+str(cluster_number))
        #Wide to logn
        df_prediction = df_prediction.reset_index().melt(id_vars =['index','cluster_id','step','method','model','RMSE']).rename(index = str, columns={'variable':'date','value':'predicted_value','index':'id'})
        true_values=true_values.reset_index().melt(id_vars=['id']).rename(index = str, columns={'variable':'date','value':'true_value'})
        ress = ress.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))
    return ress

## 1.1.1 TMRF with clusterwise prediction for GMM_Voting

In [18]:
%%time
clusters_model, cluster_per_user, clusters_parameters = clusterwise_regression(train_targets,
                                                                               test_targets,
                                                                               cluster_column = 'GMM_Voting',
                                                                               columns_wo_TS = ['clusters_r','clusters_m','clusters_f','GMM_Voting','GMM_Pair'],
                                                                               gridsearch_limit = 1)

Error per cluster:  {2.0: 0.6660010049378612, 5.0: 4.260942639521057, 0.0: 5.840328024665831, 3.0: 3.066647553627067, 8.0: 4.738625460439605, 1.0: 2.3047213634365042, 6.0: 2.8415074601268877, 9.0: 6.700348842347881}
New error:  2.761148569996368
Moved  100  users to another cluster 

Error per cluster:  {2.0: 3.277562534856147, 5.0: 1.414636231271752, 0.0: 2.1080119151382246, 3.0: 0.6965830880726239, 8.0: 0.7902808148989326, 1.0: 0.5309120948234258, 6.0: 1.1339762136566027, 9.0: 2.552892299560984}
New error:  2.7611486107020196
Error has increased. Backtrack to previous iteration
CPU times: user 2min 14s, sys: 16min 18s, total: 18min 32s
Wall time: 1min 39s


In [19]:
%%time
res = final_prediction_after_clusterwise_reg(train_targets,test_targets,step ='before_classifier',
                                      clusters = clusters_model,
                                      clusters_parameters = clusters_parameters,
                                      new_clusters_for_users = cluster_per_user,
                                      cluster_column = 'GMM_Voting',
                                      gridsearch_use = True, 
                                      columns_wo_TS = ['clusters_r','clusters_m','clusters_f','GMM_Voting','GMM_Pair'])

CPU times: user 54.9 s, sys: 5min 3s, total: 5min 58s
Wall time: 2min 18s


In [20]:
res.sample(2)

,id,cluster_id,step,method,model,RMSE,date,predicted_value,true_value
65043,2071,GMM_Voting_2.0,before_classifier,clustered_clusterwise,trmf,0.666001,1998-06-14,0.018424,0.0
10844,2335,GMM_Voting_2.0,before_classifier,clustered_clusterwise,trmf,0.666001,1998-04-30,0.062815,0.0


In [21]:
result = result.append(res)

## 1.1.2 TMRF with clusterwise prediction for GMM_Pair

In [22]:
%%time
clusters_model, cluster_per_user, clusters_parameters = clusterwise_regression(train_targets,
                                                                               test_targets,
                                                                               cluster_column = 'GMM_Pair',
                                                                               columns_wo_TS = ['clusters_r','clusters_m','clusters_f','GMM_Voting','GMM_Pair'],
                                                                               gridsearch_limit = 1)

Error per cluster:  {6.0: 0.5693927386985257, 5.0: 4.69003506533311, 3.0: 5.182696573576956, 1.0: 0.7239579630663387, 0.0: 0.57212035839337, 11.0: 0.2044735745707937, 4.0: 1.371412995240809, 9.0: 4.223893542455661, 8.0: 1.062677797047635, 10.0: 0.09169324993639597, 7.0: 0.12577274382681333, 2.0: 0.17667844678434422}
New error:  2.761148611093929
Moved  142  users to another cluster 

Error per cluster:  {6.0: 0.1040809151197133, 5.0: 1.1753888635465755, 3.0: 1.6475782725290824, 1.0: 0.08655621405566227, 0.0: 0.09379235142244916, 11.0: 0.06057623740804323, 4.0: 0.20695370806463187, 9.0: 1.6771466611574566, 8.0: 0.1169009908397308, 10.0: 9.470164297449326, 7.0: 0.06609478011967244, 2.0: 0.06846446154914672}
New error:  2.7611486074868155
Moved  422  users to another cluster 

Error per cluster:  {6.0: 0.04964834338441852, 5.0: 0.3299331250731427, 3.0: 0.5742770692368585, 1.0: 0.03993810988330365, 0.0: 0.04974407653149866, 11.0: 4.96516787522733, 4.0: 0.11328789650433282, 9.0: 0.675085065

In [23]:
%%time
res = final_prediction_after_clusterwise_reg(train_targets,test_targets,step ='before_classifier',
                                      clusters = clusters_model,
                                      clusters_parameters = clusters_parameters,
                                      new_clusters_for_users = cluster_per_user,
                                      cluster_column = 'GMM_Pair',
                                             gridsearch_use = True, 
                                      columns_wo_TS = ['clusters_r','clusters_m','clusters_f','GMM_Voting','GMM_Pair'])

CPU times: user 30.5 s, sys: 2min 56s, total: 3min 27s
Wall time: 1min 47s


In [24]:
res.sample(2)

,id,cluster_id,step,method,model,RMSE,date,predicted_value,true_value
431,12237,GMM_Pair_11.0,before_classifier,clustered_clusterwise,trmf,4.965168,1998-04-22,0.070021,0.0
23743,9869,GMM_Pair_11.0,before_classifier,clustered_clusterwise,trmf,4.965168,1998-06-08,1.487941,0.0


In [25]:
result = result.append(res)

## 1.2.1 Theta for GMM_Voting (ids are not moved!)

In [26]:
%%time
res = theta_prediction(train_targets,test_targets,
                 step ='before_classifier',
                 cluster_column = 'GMM_Voting',
                 columns_wo_TS = ['clusters_r','clusters_m','clusters_f','GMM_Voting','GMM_Pair'])
result = result.append(res)

CPU times: user 28.4 s, sys: 0 ns, total: 28.4 s
Wall time: 37.9 s


## 1.2.2 Theta for GMM_Voting (ids are not moved!)

In [27]:
%%time
res = theta_prediction(train_targets,test_targets,
                 step ='before_classifier',
                 cluster_column = 'GMM_Pair',
                 columns_wo_TS = ['clusters_r','clusters_m','clusters_f','GMM_Voting','GMM_Pair'])
result = result.append(res)

CPU times: user 28.7 s, sys: 0 ns, total: 28.7 s
Wall time: 32.7 s


## 2.0 Training the classifier to predict cluster (ids are not moved!)

In [28]:
predicted_clusters = {}
columns_wo_TS = ['clusters_r','clusters_m','clusters_f','GMM_Voting','GMM_Pair']

### 2.1 Prediction of clusters for the 1st possible cluster column

In [29]:
cluster_column = 'GMM_Voting'

In [30]:
# use the classifier on the same length as the length used for training (all minus 70)
# Data for training Classifier
X_2000_train,test_2000 = train_test_split(df_train.transpose(), test_size=.127, shuffle=False)
X_2000_train = X_2000_train.transpose()
test_2000 = test_2000.transpose()
# Predict clusters using these data
X_600_train, test_600= train_test_split(df_test.transpose(), test_size=.127, shuffle=False)
X_600_train = X_600_train.transpose()
test_600.transpose()
y_train_2000 = df_train[cluster_column]
X_2000_train = X_2000_train.drop(columns = columns_wo_TS)

# Grid SearchCV
from xgboost import XGBClassifier 
params = {
        'min_child_weight': [1, 7],
        'gamma': [0.5,4],
        'subsample': [0.7,0.9],# 0.8, 1.0],
        'max_depth': [5]
        }
clf = XGBClassifier(seed=0, silent=True)
clf = GridSearchCV(clf,params,cv=3,n_jobs=-1,verbose=2)

In [31]:
%%time
clf.fit(X_2000_train,y_train_2000)
clf.best_estimator_
# 12 fits - 30 seconds
# 108 fits - 
xgb =clf.best_estimator_

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  24 | elapsed:   43.0s remaining:  1.0min
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  1.2min finished


CPU times: user 38.7 s, sys: 0 ns, total: 38.7 s
Wall time: 2min 17s


In [32]:
%%time
xgb.fit(X_2000_train.values,y_train_2000.values)
predicted_clusters[cluster_column] = xgb.predict(X_600_train.values)

CPU times: user 36.9 s, sys: 0 ns, total: 36.9 s
Wall time: 58.7 s


### 2.2 Prediction of clusters for the 2nd possible cluster column

In [33]:
cluster_column = 'GMM_Pair'

In [34]:
# use the classifier on the same length as the length used for training (all minus 70)
# Data for training Classifier
X_2000_train,test_2000 = train_test_split(df_train.transpose(), test_size=.127, shuffle=False)
X_2000_train = X_2000_train.transpose()
test_2000 = test_2000.transpose()
# Predict clusters using these data
X_600_train, test_600= train_test_split(df_test.transpose(), test_size=.127, shuffle=False)
X_600_train = X_600_train.transpose()
test_600.transpose()
y_train_2000 = df_train[cluster_column]
X_2000_train = X_2000_train.drop(columns = columns_wo_TS)

# Grid SearchCV
from xgboost import XGBClassifier 
params = {
        'min_child_weight': [1, 7],
        'gamma': [0.5,4],
        'subsample': [0.7,0.9],# 0.8, 1.0],
        'max_depth': [5]
        }
clf = XGBClassifier(seed=0, silent=True)
clf = GridSearchCV(clf,params,cv=3,n_jobs=-1,verbose=2)

In [35]:
%%time
clf.fit(X_2000_train,y_train_2000)
clf.best_estimator_
# 12 fits - 30 seconds
xgb =clf.best_estimator_

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  24 | elapsed:   20.0s remaining:   28.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   42.6s finished


CPU times: user 36.2 s, sys: 0 ns, total: 36.2 s
Wall time: 1min 19s


In [36]:
%%time
xgb.fit(X_2000_train.values,y_train_2000.values)
predicted_clusters[cluster_column] = xgb.predict(X_600_train.values)

CPU times: user 39.2 s, sys: 0 ns, total: 39.2 s
Wall time: 39.3 s


### 2.9 Add predictions to dataframe 

In [37]:
for k,v in predicted_clusters.items():
    df_test.insert(0,k,v)

## 3.0 Prediction with predicted clusters on full data (2600 rows)

In [38]:
# Add additional users on top of starting 2000 
df_with_predicted_labels = df_train.append(df_test)

In [39]:
train_targets, test_targets= train_test_split(df_with_predicted_labels.transpose(), test_size=.127, shuffle=False)
train_targets = train_targets.transpose()
test_targets = test_targets.transpose()

## 3.1 TRMF

## 3.1.1 TMRF with clusterwise prediction for GMM_Voting

In [40]:
%%time
clusters_model, cluster_per_user, clusters_parameters = clusterwise_regression(train_targets,
                                                                               test_targets,
                                                                               cluster_column = 'GMM_Voting',
                                                                               columns_wo_TS = ['clusters_r','clusters_m','clusters_f','GMM_Voting','GMM_Pair'],
                                                                               gridsearch_limit = 1)

Error per cluster:  {2.0: 1.9082018650721824, 5.0: 4.441425255703498, 0.0: 6.417924412863097, 3.0: 5.319232471586175, 8.0: 5.670557121075327, 1.0: 4.448359251295415, 6.0: 5.482681270075262, 9.0: 6.85678107909232}
New error:  2.6428809076008424
Moved  178  users to another cluster 

Error per cluster:  {2.0: 2.7289688222470225, 5.0: 1.6941887600955277, 0.0: 2.614854096477642, 3.0: 1.6914299406696114, 8.0: 1.3862871272126336, 1.0: 1.4904148466622271, 6.0: 1.8459910814322251, 9.0: 2.369293061196182}
New error:  2.6428808653256795
Moved  854  users to another cluster 

Error per cluster:  {2.0: 0.38633683066620583, 5.0: 0.6464880621146304, 0.0: 0.9477157348198544, 3.0: 0.5530669060761029, 8.0: 7.365807460381722, 1.0: 0.43029552011881717, 6.0: 0.889632239857322, 9.0: 1.0718204442821748}
New error:  2.6428808412241747
Moved  367  users to another cluster 

Error per cluster:  {2.0: 2.5517646126240607, 5.0: 0.35761199566102597, 0.0: 0.46515382239355907, 3.0: 0.25084632094759474, 8.0: 3.657911

In [41]:
%%time
res = final_prediction_after_clusterwise_reg(train_targets,test_targets,step ='after_classifier',
                                      clusters = clusters_model,
                                      clusters_parameters = clusters_parameters,
                                      new_clusters_for_users = cluster_per_user,
                                      cluster_column = 'GMM_Voting',
                                             gridsearch_use = True, 
                                      columns_wo_TS = ['clusters_r','clusters_m','clusters_f','GMM_Voting','GMM_Pair'])

CPU times: user 2min 43s, sys: 14min 10s, total: 16min 53s
Wall time: 5min 22s


In [42]:
res.sample(2)

,id,cluster_id,step,method,model,RMSE,date,predicted_value,true_value
81772,15750,GMM_Voting_8.0,after_classifier,clustered_clusterwise,trmf,7.365807,1998-06-25,0.306702,0.0
491692,10148,GMM_Voting_2.0,after_classifier,clustered_clusterwise,trmf,0.386337,1998-06-20,0.030189,0.0


In [43]:
result = result.append(res)

## 3.1.2 TMRF with clusterwise prediction for GMM_Pair

In [44]:
%%time
clusters_model, cluster_per_user, clusters_parameters = clusterwise_regression(train_targets,
                                                                               test_targets,
                                                                               cluster_column = 'GMM_Pair',
                                                                               columns_wo_TS = ['clusters_r','clusters_m','clusters_f','GMM_Voting','GMM_Pair'],
                                                                               gridsearch_limit = 1)

Error per cluster:  {6.0: 1.6753176783468933, 5.0: 4.50155562490698, 3.0: 6.055327697789854, 1.0: 0.7239579630663387, 0.0: 2.0276654997335655, 11.0: 0.2044735745707937, 4.0: 1.9369544471357756, 9.0: 5.930651322984639, 8.0: 1.062677797047635, 10.0: 0.09169324993639597, 7.0: 0.12577274382681333, 2.0: 0.2126299683427585}
New error:  2.6428808279812745
Moved  793  users to another cluster 

Error per cluster:  {6.0: 0.2543665372516082, 5.0: 1.1882537245656455, 3.0: 1.9469218852128456, 1.0: 0.08655621405566227, 0.0: 0.2519481507761946, 11.0: 0.06057623740804323, 4.0: 0.3801780026311857, 9.0: 2.3479026451943446, 8.0: 0.1169009908397308, 10.0: 9.015926090450913, 7.0: 0.06609478011967244, 2.0: 0.0729962752807712}
New error:  2.6428808753174833
Error has increased. Backtrack to previous iteration
CPU times: user 4min 23s, sys: 27min 7s, total: 31min 30s
Wall time: 4min 1s


In [45]:
%%time
res = final_prediction_after_clusterwise_reg(train_targets,test_targets,step ='after_classifier',
                                      clusters = clusters_model,
                                      clusters_parameters = clusters_parameters,
                                      new_clusters_for_users = cluster_per_user,
                                      cluster_column = 'GMM_Pair',
                                             gridsearch_use = True, 
                                      columns_wo_TS = ['clusters_r','clusters_m','clusters_f','GMM_Voting','GMM_Pair'])

CPU times: user 2min 7s, sys: 13min 56s, total: 16min 4s
Wall time: 1min 37s


In [46]:
res.sample(2)

,id,cluster_id,step,method,model,RMSE,date,predicted_value,true_value
252535,21763,GMM_Pair_6.0,after_classifier,clustered_clusterwise,trmf,1.675318,1998-05-28,0.258445,0.0
405305,192,GMM_Pair_6.0,after_classifier,clustered_clusterwise,trmf,1.675318,1998-06-19,0.336618,0.0


In [47]:
result = result.append(res)

## 3.2.1 Theta for GMM_Voting (ids are not moved!)

In [48]:
%%time
res = theta_prediction(train_targets,test_targets,
                 step ='after_classifier',
                 cluster_column = 'GMM_Voting',
                 columns_wo_TS = ['clusters_r','clusters_m','clusters_f','GMM_Voting','GMM_Pair'])
result = result.append(res)

CPU times: user 1min 41s, sys: 36 ms, total: 1min 41s
Wall time: 1min 41s


## 3.2.2 Theta for GMM_Voting (ids are not moved!)

In [49]:
%%time
res = theta_prediction(train_targets,test_targets,
                 step ='after_classifier',
                 cluster_column = 'GMM_Pair',
                 columns_wo_TS = ['clusters_r','clusters_m','clusters_f','GMM_Voting','GMM_Pair'])

CPU times: user 1min 39s, sys: 0 ns, total: 1min 39s
Wall time: 1min 39s


In [53]:
res.cluster_id.unique()

array(['GMM_Pair_6.0', 'GMM_Pair_5.0', 'GMM_Pair_3.0', 'GMM_Pair_1.0',
       'GMM_Pair_0.0', 'GMM_Pair_11.0', 'GMM_Pair_4.0', 'GMM_Pair_9.0',
       'GMM_Pair_8.0', 'GMM_Pair_10.0', 'GMM_Pair_7.0', 'GMM_Pair_2.0'],
      dtype=object)

In [50]:
result = result.append(res)

### Save to file 

In [51]:
# On both predicted_values and true_value the normalizatin have to be performed
result.to_csv(prediction_path)
result.sample(2)

,id,cluster_id,step,method,model,RMSE,date,predicted_value,true_value
287450,9301,GMM_Pair_6.0,after_classifier,clustered_clusterwise,trmf,1.675318,1998-06-02,0.030189,0.0
37305,1855,GMM_Voting_2.0,before_classifier,clustered,theta,0.655307,1998-05-22,-0.000616,0.0
